### Load libs

In [1]:
import requests
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime

n = 200
pd.set_option('display.max_rows', n)
pd.set_option('display.max_columns', n)


In [2]:
today = str(datetime.now())[0:10]


### Set credentials and project

In [3]:
# access token
access_token = 'xxxxxxxxxxx'


In [4]:
# type of stocks
stocks_name = 'Tech'


In [5]:
# set tickers
tickers = ['AAPL', 'MSFT', 'GOOG']


### Create functions

In [6]:
def get_info(ticker, access_token):
    
    print('getting messages for {}...'.format(ticker))
    
    
    """
    Function to parse stocktwits json formatted data into df
    """
    
    # make request
    header = {'Authorization': 'Bearer {}'.format(access_token)}
    company_url = 'https://api.stocktwits.com/api/2/streams/symbol/{}.json'.format(ticker)
    response = requests.get(company_url, headers = header).json()
    
    # get messages
    messages = response['messages'] if 'messages' in response else ''
    
    # parse messages
    holder = []
    for message in messages:

        user = message['user'] if 'user' in message else ''
        likes = message['likes'] if 'likes' in message else ''
        entities = message['entities'] if 'entities' in message else ''
        
        message_dict = {}
        message_dict['id'] = message['id'] if 'id' in message else ''
        message_dict['body'] = message['body'] if 'body' in message else ''
        message_dict['created_at'] = message['created_at'] if 'created_at' in message else ''
        message_dict['user_id'] = user['id'] if 'id' in user else ''
        message_dict['join_date'] = user['join_date'] if 'join_date' in user else ''
        message_dict['followers'] = user['followers'] if 'followers' in user else ''
        message_dict['following'] = user['following'] if 'following' in user else ''
        message_dict['likes'] = likes['total'] if 'total' in likes else ''
        try:
            message_dict['sentiment'] = entities['sentiment']['basic'] if 'basic' in entities['sentiment'] and 'sentiment' in entities else ''  
        except:
            message_dict['sentiment'] = ''
        
        holder.append(message_dict)
        
    # return df
    columns = ['id', 'body', 'sentiment', 'created_at', 'user_id', 'join_date', 'followers', 'following', 'likes']
    df = pd.DataFrame(holder, columns = columns)
    
    print('complete!')
    
    return df


In [7]:
analyser = SentimentIntensityAnalyzer()
def get_vader_sent(text):
    
    """
    Get vader sentiment from text.
    """
    
    text = str(text)
    responses = analyser.polarity_scores(text)
    sent = responses['compound']
    
    return sent


In [8]:
def vader_df(row, column):
    
    """
    Apply vader to df.
    """
    
    text = str(row[column])
    
    sent = get_vader_sent(text)
    
    return sent


### Apply functions on stocks data

In [9]:
master_df = pd.DataFrame()

for ticker in tickers:
    
    df = get_info(ticker, access_token)
    
    master_df = master_df.append(df).reset_index(drop = True)
    
    print('------------------------------------')
    
    

getting messages for AAPL...
complete!
------------------------------------
getting messages for MSFT...
complete!
------------------------------------
getting messages for GOOG...
complete!
------------------------------------


In [10]:
# apply sentiment to df
master_df['sentiment_score'] = master_df.apply(vader_df, args = ('body', ), axis = 1)


In [11]:
master_df

id                                               body sentiment  \
0   192269529                           $AAPL IV BOUT TO EATTTTT             
1   192269525  $BABA $AAPL Called the move, if this makes new...             
2   192269516                                  $AAPL Merica 😅💰🇺🇸             
3   192269507  $AAPL How epic for this to open below $300 tom...   Bearish   
4   192269504  $aapl  yep,  the master does it again \n\nlive...             
5   192269486                     $AAPL  bulls are so stupid lol   Bearish   
6   192269484                           $AAPL omg shorts ... lol             
7   192269480  $AAPL sweet sold iron condors 305/302.5P &amp;...             
8   192269475                       $AAPL  tanking - wait for me   Bearish   
9   192269466  $AAPL if apple beat and it&#39;s up less after...             
10  192269459  $AAPL RESULTS WERE NOT GOOD ENOUGH FOR CURRENT...             
11  192269453                      $AAPL don’t cry when it fades             
12  192269445               $AMD $AAPL \n\nGame over short term.             
13  192269442  $AAPL Now...don&#39;t go up too much, My $qqq ...             
14  192269435            $AAPL | WUHAN epidemic - more more more   Bearish   
15  192269421  $AAPL one analyst says APPLE will double  serv...             
16  192269414                   $SPY  was in meetings $aapl  wow             
17  192269411                                   $AAPL Fade this.   Bearish   
18  192269410  $AAPL It&#39;s all about China and there is no...             
19  192269404                        $AAPL damn.  Some big sales             
20  192269403  $AAPL don’t think big enough best to hold the ...   Bearish   
21  192269398                          $AAPL here comes the drop   Bearish   
22  192269394  $AAPL This pig has priced in 10 years of PERFE...             
23  192269380  #Apple Surges After Smashing Expectations, Pro...             
24  192269371          $AAPL easy short position.  Cover at 310.   Bearish   
25  192269362  $AAPL just curious to know how many shares peo...             
26  192269357  $AAPL Love it when bears think shorts covering...   Bullish   
27  192269351  $FB $MSFT Higher high   \n \nWhen $AAPL is goo...             
28  192269350  Apple&#39;s Q1 Earnings Boosted By Strong iPho...             
29  192269341                             $AAPL again I say $325             
30  192269456  $SPY Bears have to sweat all day tomorrow for ...             
31  192269351  $FB $MSFT Higher high   \n \nWhen $AAPL is goo...             
32  192269126  $MSFT Thank you Father Market for those cheap ...   Bullish   
33  192268961  .Bears are doing or soon will be doing this\n$...   Bullish   
34  192268426  $MSFT $AAPL $FB $TSLA $BA CORONA WHAT? MARKET ...   Bullish   
35  192268063  $TVIX $MSFT tomorrow 🤩\n$BA tomorrow- not sure...             
36  192267950  $MSFT All eyes on this now.... If nothing come...   Bullish   
37  192267821  $MSFT shes starting to already with profit tak...   Bullish   
38  192267524  $MSFT going to be a lot of profit takers from ...   Bullish   
39  192266840  $MSFT now our turn tomorrow. We fly fly . 167....   Bullish   
40  192266243  $AAPL $SPY $MSFT \nFUTES RIPPINNNNNN 🚀🚀🚀🚀🚀🖕🏻🖕🏻...   Bullish   
41  192265986                 $AAPL ok time for $MSFT to deliver   Bullish   
42  192265786  $AAPL INTO THE NEXT $MSFT $AMZN $GOOGL ALL WIL...   Bullish   
43  192265304  What Lies Ahead for Microsoft ETFs This Earnin...             
44  192265138  $MSFT king of all beating is for this tomorrow...   Bullish   
45  192264138                             $AAPL $AMD $MSFT $AMZN             
46  192264122                        $MSFT so nice thanks apple!   Bullish   
47  192263907  $AAPL so Sweet! \n \nMy sister is GOOD !! \nNE...             
48  192263735         $AAPL $TSLA $BYND $MSFT $COST\n\nRIPPIN!!!   Bullish   
49  192263574  $MSFT $AAPL beat left and right!! We go up tom...   Bullish   
50  192263036  $MSFT anyone else 

### Export results

In [12]:
# set export
folder = 'data/'
file = '{} messages {}.xlsx'.format(stocks_name, today)
path = folder + file
path


'data/Tech messags 2020-01-28.xlsx'

In [13]:
# export
master_df.to_excel(path, encoding='utf8', index = False)


In [14]:
print('complete...')

complete...


### End 